In [0]:
-- ticket medio
SELECT
  AVG(valor_pedido) AS ticket_medio
FROM (
  SELECT
    order_id,
    SUM(price) AS valor_pedido
  FROM workspace.projeto_vendas_bronze.order_items
  GROUP BY order_id
) t; -- tabela temporária

In [0]:
-- Faturamento por estado
SELECT
  customer_state,
  SUM(valor_pedido) AS faturamento_estado
FROM (
  SELECT
    o.order_id,
    c.customer_state,
    SUM(oi.price) AS valor_pedido
  FROM workspace.projeto_vendas_bronze.order_items oi
  JOIN workspace.projeto_vendas_bronze.orders o
    ON oi.order_id = o.order_id
  JOIN workspace.projeto_vendas_bronze.customers c
    ON o.customer_id = c.customer_id
  GROUP BY
    o.order_id, c.customer_state
) t
GROUP BY customer_state
ORDER BY faturamento_estado DESC;

In [0]:
-- Pedidos acima do ticket médio

SELECT
  order_id,
  SUM(price) AS valor_pedido
FROM workspace.projeto_vendas_bronze.order_items
GROUP BY order_id
HAVING SUM(price) > (
  SELECT
    AVG(valor_pedido)
  FROM (
    SELECT
      order_id,
      SUM(price) AS valor_pedido
    FROM workspace.projeto_vendas_bronze.order_items
    GROUP BY order_id
  ) t
);

In [0]:
-- Categorias acima da média de faturamento
SELECT
  product_category_name,
  SUM(oi.price) AS faturamento
FROM workspace.projeto_vendas_bronze.order_items oi
JOIN workspace.projeto_vendas_bronze.products p
  ON oi.product_id = p.product_id
GROUP BY product_category_name
HAVING SUM(oi.price) > (
  SELECT
    AVG(faturamento_categoria)
  FROM (
    SELECT
      p.product_category_name,
      SUM(oi.price) AS faturamento_categoria
    FROM workspace.projeto_vendas_bronze.order_items oi
    JOIN workspace.projeto_vendas_bronze.products p
      ON oi.product_id = p.product_id
    GROUP BY p.product_category_name
  ) t
);   

## Refatorando com CTE

In [0]:
WITH pedidos AS (
  SELECT
    order_id,
    SUM(price) AS valor_pedido
  FROM workspace.projeto_vendas_bronze.order_items
  GROUP BY order_id
)

SELECT AVG(valor_pedido) AS ticket_medio FROM pedidos;

In [0]:
WITH pedidos AS (
  SELECT o.order_id,
         c.customer_state,
         SUM(oi.price) AS valor_pedido
  FROM workspace.projeto_vendas_bronze.order_items oi
  JOIN workspace.projeto_vendas_bronze.orders o
    ON oi.order_id = o.order_id
  JOIN workspace.projeto_vendas_bronze.customers c
    ON o.customer_id = c.customer_id
  WHERE o.order_status = 'delivered'
  GROUP BY o.order_id, c.customer_state
)

SELECT
  customer_state,
  SUM(valor_pedido) AS faturamento
FROM pedidos
GROUP BY customer_state
ORDER BY faturamento DESC;